In [8]:
from MOT import MOT
import numpy as np
import matplotlib.pyplot as plt
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
# from open3d import JVisualizer 
from Utils import *
from DDBSCAN import Raster_DBSCAN
import open3d as op3
op3.visualization.webrtc_server.enable_webrtc()
from open3d.web_visualizer import draw


[Open3D INFO] WebRTC GUI backend enabled.


RuntimeError: Open3D WebVisualizer Jupyter extension is not available. To use WebVisualizer, build Open3D with -DBUILD_JUPYTER_EXTENSION=ON.

In [2]:
def get_pcd_thru_td(Td_map):
    Xs = []
    Ys = []
    Zs = []
    for i in range(Td_map.shape[0]):
        longitudes = theta[i]*np.pi / 180
        latitudes = azimuths * np.pi / 180 
        hypotenuses = Td_map[i] * np.cos(longitudes)
        X = hypotenuses * np.sin(latitudes)
        Y = hypotenuses * np.cos(latitudes)
        Z = Td_map[i] * np.sin(longitudes)
        Xs.append(X)
        Ys.append(Y)
        Zs.append(Z)

    Xs = np.concatenate(Xs)
    Ys = np.concatenate(Ys)
    Zs = np.concatenate(Zs)
    pcd = op3.geometry.PointCloud()
    XYZ = np.concatenate([Xs.reshape(-1,1),Ys.reshape(-1,1),Zs.reshape(-1,1)],axis = 1)
    pcd.points = op3.utility.Vector3dVector(XYZ)
    
    return pcd 

'/Users/czhui960/Documents/Lidar/Lidar_Code'

In [4]:
# os.chdir(r'/Users/czhui960/Documents/Lidar/RawLidarData/FrameSamplingTest')
# thred_map = np.load(r'Output File/thred_map_1200.npy')
collector = RansacCollector(pcap_path=r'../RawLidarData/Veteran/Veteran.pcap',output_file_path='../MergingResult/',update_frame_num=2000)
collector.gen_tdmap()
collector.gen_thredmap(d = 1.4,thred_s = 0.7,N = 20,delta_thred = 1e-3,step = 0.01)

  0%|          | 2/2000 [00:00<02:01, 16.51it/s]

Loading pcap...


100%|██████████| 2000/2000 [01:34<00:00, 21.27it/s]


Generating Threshold Map


In [10]:
Td_map = collector.aggregated_map[3]

In [9]:
db = Raster_DBSCAN(window_size=(5,11),eps = 1.8, min_samples= 15,Td_map_szie=collector.thred_map.shape)   

In [11]:
Foreground_map = (Td_map < collector.thred_map)&(Td_map != 0)
Labeling_map = db.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)

In [14]:
Labeling_map

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])